In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import glob
import PIL
import webdataset as wds
import os
import os.path
import io
from ocrlib import extract_seg
from random import randrange

In [28]:
allpages = glob.glob(base+"ENGLISH/LINEWORD/IMAGEBIN/*.TIF")
allpages = [os.path.basename(x) for x in allpages]
allpages = [re.sub("BIN.TIF$", "", x) for x in allpages]
allpages = sorted(allpages)
print(len(allpages), allpages[:3])

1600 ['A001', 'A002', 'A003']


In [46]:
def readfile(fname, mode=""):
    if os.path.exists(fname):
        with open(fname, "r"+mode) as stream:
            return stream.read()
    else:
        if "uw1-1" not in fname:
            print(f"{fname} not found", file=sys.stderr)
        return None
def readbin(fname, mode=""):
    return readfile(fname, mode="b")

In [47]:
output = wds.TarWriter("uw3-original.tar")

base = "/mdata/IUPR/uw3/"
lineword = base+"ENGLISH/LINEWORD/"
base1 = "/mdata/IUPR/uw1-1/"
real = base1+"ENGLISH/JOURNAL/REAL/"

allmeta = []
for i, page in enumerate(allpages):
    if i%100==0: print(i, page)
    p = {}
    p["__key__"] = page
    # data from UW3
    p["image.tif"] = readbin(lineword+"IMAGEBIN/"+page+"BIN.TIF")
    p["linebox.txt"] = readfile(lineword+"LINEBOXS/"+page+"LINE.BOX")
    p["wordbox.txt"] = readfile(lineword+"WORDBOXS/"+page+"WORD.BOX")
    p["zonebox.txt"] = readfile(lineword+"ZONEBOXS/"+page+"ZONE.BOX")
    p["dafs"] = readbin(lineword+"DAFS/"+page+".DAF")
    p["dafs.txt"] = readfile(lineword+"DAFS/"+page+".TXT")
    p["labzones.txt"] = readfile(lineword+"LABZONES/"+page+"LABZONES.txt")
    # data from UW1-1
    p["1pageattr.txt"] = readfile(real+"PAGEATTR/"+page+"PAGE.ATT")
    p["1pageboxs.txt"] = readfile(real+"PAGEBOXS/"+page+"PAGE.BOX")
    p["1zoneattr.txt"] = readfile(real+"ZONEATTR/"+page+"ZONE.ATT")
    p["1zoneboxs.txt"] = readfile(real+"ZONEBOXS/"+page+"ZONE.BOX")
    p["1pageattr.txt"] = readfile(real+"PAGEATTR/"+page+"PAGE.ATT")
    p["1pageboxs.txt"] = readfile(real+"PAGEBOXS/"+page+"PAGE.BOX")
    p["1ground.txt"] = readfile(real+"GROUND/"+page+"GRND.TRU")
    p["1cond.txt"] = readfile(real+"PAGECOND/"+page+"PAGE.CON")
    missing = [k for k, v in p.items() if v is None]
    for k in missing: del p[k]
    output.write(p)
    
output.close()

0 A001
100 A063
200 D038
300 E009
400 E048
500 H048
600 IG0F
700 K00F
800 N04L
900 S02N
1000 W020
1100 W0C3
1200 W0L3
1300 W114
1400 W1B1
1500 W1K2


In [83]:
output = wds.TarWriter("uw3-hocr.tar")

base = "/mdata/IUPR/uw3/"
lineword = base+"ENGLISH/LINEWORD/"
base1 = "/mdata/IUPR/uw1-1/"
real = base1+"ENGLISH/JOURNAL/REAL/"

allmeta = []
for i, page in enumerate(allpages):
    if i%100==0: print(i, page)
    p = {}
    p["__key__"] = page
    # data from UW3
    image = PIL.Image.open(lineword+"IMAGEBIN/"+page+"BIN.TIF")
    jpg = io.BytesIO()
    image.save(jpg, format="JPEG", quality=100)
    p["page.jpg"] = jpg.getvalue()
    p["hocr.html"] = readfile(base+"hocr/uw3/gt-uw3/"+page+".html")
    output.write(p)
    
output.close()

0 A001
100 A063
200 D038
300 E009
400 E048
500 H048
600 IG0F
700 K00F
800 N04L
900 S02N
1000 W020
1100 W0C3
1200 W0L3
1300 W114
1400 W1B1
1500 W1K2
